In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./data/ThorLabs_Lenses.csv').iloc[:, :-1]

def clean_value(val):
    if not isinstance(val, str):
        return val
    if 'mm' in val:
        return float(val.replace(' mm', ''))
    if '"' in val:
        val = val.replace('"', '')
        return float(eval(val)) * 25.4 if '/' in val else float(val) * 25.4
    return val

unit_columns = ['Diameter', 'Focal Length', 'Radius of Curvature', 
                'Center Thickness', 'Edge Thickness', 'Back Focal Length']

for col in unit_columns:
    df[col] = df[col].apply(clean_value)
    df = df.rename(columns={col: f'{col} (mm)'})

df = df.rename(columns={'Item #a': 'Item #', 'Diopterb': 'Diopter'})
# df

# Override existing file
df.to_csv('./data/ThorLabs_Lenses.csv', index=False)

In [22]:
# Unique radii
rad_ll = np.array(df['Diameter (mm)'].unique()) / 2
print("Radii (mm):", rad_ll)

Radii (mm): [ 1.    1.5   2.5   3.    6.35 12.7  25.4  37.5 ]


# Finding Params for Lens 1

In [23]:
z_0 = 11.01 # mm

# Optimal focal lengths for 33 degree divergence
optimal_f = np.array([ rad/np.tan(np.deg2rad(33)) for rad in rad_ll ])
print("Optimal Focal Lengths (mm):", optimal_f)

# plt.figure(figsize=(10, 6))
# plt.scatter(rad_ll, optimal_f, color='blue', label='Optimal Focal Lengths')
# plt.plot(rad_ll, optimal_f, linestyle='--', color='lightblue')
# plt.xlabel('Radius (mm)')
# plt.ylabel('Optimal Focal Length (mm)')
# plt.title('Optimal Focal Length vs. Radius for 33° Divergence')
# plt.grid(True)
# plt.legend()
# plt.show()

# Finding the closest available focal lengths in the dataset
available_f = df['Focal Length (mm)'].values
closest_f = np.array([ min(available_f, key=lambda x: abs(x - opt_f)) for opt_f in optimal_f ])

# Remove items that are smaller than z_0
closest_f = closest_f[closest_f >= z_0]
print("Closest Focal Lengths (mm):", closest_f)

# Finding all lens Item # for these focal lengths
lens_items = [df[df['Focal Length (mm)'] == f]['Item #'].tolist() for f in closest_f]
# Convert list of lists to a flat list and remove duplicates
lens_items = list(set(item for sublist in lens_items for item in sublist))
print("Lens Item Numbers:", lens_items)

# z values for these focal lengths
z_values = closest_f - z_0
print("z values (mm):", z_values)

Optimal Focal Lengths (mm): [ 1.53986496  2.30979745  3.84966241  4.61959489  9.77814252 19.55628504
 39.11257008 57.74493614]
Closest Focal Lengths (mm): [20.1 40.1 60. ]
Lens Item Numbers: ['LA4130', 'LA4042', 'LA4647', 'LA4306', 'LA4194', 'LA4022']
z values (mm): [ 9.09 29.09 48.99]


In [24]:
# Fetch data for lens_items from the dataframe as a dataframe
lens_data = df[df['Item #'].isin(lens_items)]
lens_data = lens_data.sort_values(by='Diameter (mm)')
lens_data.reset_index(drop=True, inplace=True)
lens_data

,Item #,Diameter (mm),Focal Length (mm),Diopter,Radius of Curvature (mm),Center Thickness (mm),Edge Thickness (mm),Back Focal Length (mm)
0,LA4194,6.0,20.1,49.8,9.2,2.0,1.5,18.7
1,LA4647,12.7,20.1,49.8,9.2,4.3,1.8,17.1
2,LA4130,12.7,40.1,24.9,18.4,2.9,1.8,38.1
3,LA4042,12.7,60.0,16.7,27.5,2.5,1.8,58.2
4,LA4306,25.4,40.1,24.9,18.4,7.1,2.0,35.3
5,LA4022,25.4,60.0,16.7,27.5,5.1,2.0,56.5


In [ ]:
# Save lens data to CSV
lens_data.to_csv('./data/l1_candidates.csv', index=False)

# Finding Params for Lens 2

In [26]:
acceptance_angle = np.deg2rad(12.4)  # 33 degrees in radians

# Find z for each lens
# z = radius / tan(acceptance_angle)
z_values = rad_ll / np.tan(acceptance_angle)
print("z values (mm):", z_values)

optimal_f = z_values

# Finding the closest available focal lengths in the dataset
available_f = df['Focal Length (mm)'].values
closest_f = np.array([ min(available_f, key=lambda x: abs(x - opt_f)) for opt_f in optimal_f ])

# Finding all lens Item # for these focal lengths
lens_items = [df[df['Focal Length (mm)'] == f]['Item #'].tolist() for f in closest_f]
# Convert list of lists to a flat list and remove duplicates
lens_items = list(set(item for sublist in lens_items for item in sublist))
print("Lens Item Numbers:", lens_items)

z values (mm): [  4.54826083   6.82239125  11.37065209  13.6447825   28.8814563
  57.7629126  115.5258252  170.55978129]
Lens Item Numbers: ['LA4021', 'LA4034', 'LA4022', 'LA4043', 'LA4280', 'LA4033', 'LA4042', 'LA4036', 'LA4026', 'LA4001', 'LA4024', 'LA4005', 'LA4249']


In [27]:
# Fetch data for lens_items from the dataframe as a dataframe
lens_data = df[df['Item #'].isin(lens_items)]
lens_data = lens_data.sort_values(by='Diameter (mm)')
lens_data.reset_index(drop=True, inplace=True)
lens_data

,Item #,Diameter (mm),Focal Length (mm),Diopter,Radius of Curvature (mm),Center Thickness (mm),Edge Thickness (mm),Back Focal Length (mm)
0,LA4024,2.0,4.0,250.0,1.8,1.0,0.6,3.31
1,LA4026,2.0,6.0,166.7,2.8,1.0,0.7,5.31
2,LA4036,3.0,6.0,166.7,2.8,1.5,1.0,4.97
3,LA4249,5.0,10.0,100.0,4.6,2.2,1.5,8.50
4,LA4280,6.0,10.0,100.0,4.6,2.6,1.5,8.20
5,LA4001,12.7,15.0,66.7,6.9,6.0,1.8,10.90
6,LA4042,12.7,60.0,16.7,27.5,2.5,1.8,58.20
7,LA4043,12.7,125.0,8.0,57.3,2.2,1.8,123.50
8,LA4005,12.7,175.0,5.7,80.2,2.1,1.8,173.60
9,LA4021,25.4,30.0,33.3,13.8,10.5,2.0,22.80


In [ ]:
# Save lens data to CSV
lens_data.to_csv('./data/l2_candidates.csv', index=False)